In [1]:
(defn swap [v i1 i2] 
   (assoc v i2 (v i1) i1 (v i2)))

(def goal [8 7 6 5 4 3 2 1 0])

(defn move_up [state] 
    (let [empty_pos (.indexOf state 0)]
        (cond
            (some #(= empty_pos %) [6 7 8]) nil
            :else (swap state empty_pos (+ empty_pos 3))
            )
        ))
(defn move_down [state] 
    (let [empty_pos (.indexOf state 0)]
        (cond
            (some #(= empty_pos %) [0 1 2]) nil
            :else (swap state empty_pos (- empty_pos 3))
            )
        ))
(defn move_left [state] 
    (let [empty_pos (.indexOf state 0)]
        (cond
            (some #(= empty_pos %) [2 5 8]) nil
            :else (swap state empty_pos (+ empty_pos 1))
            )
        ))
(defn move_right [state] 
    (let [empty_pos (.indexOf state 0)]
        (cond
            (some #(= empty_pos %) [0 3 6]) nil
            :else (swap state empty_pos (- empty_pos 1))
            )
        ))

(defn get_successors [state]
    (vec (filter (fn [e] (not (nil? e))) [(move_up state) (move_down state) (move_left state) (move_right state)]))
    )

(defn solved [state]
    (= state goal))

(defn h1 [state] 
    (apply + (vec (map (fn [a b] (if (= a b) 0 1)) state goal))))
(defn h2 [state] 
    (apply + (vec (map (fn [a b] (if (> a b) (- a b) (- b a))) state goal))))

(defn get_random_state [swaps] 
    ((fn bfs [initial_state]
    ((fn bfs_in [states marked swaps] 
    (let [worst (apply max-key (fn [s] (- (h1 s))) states)]
        (if (= swaps 0) worst 
            (
                let [successors (get_successors worst)] 
                    (bfs_in
                        (vec (filter (fn [e] (every? #(not= e %) marked)) (vec (concat successors (vec (remove (fn [e] (= e worst)) states))))))
                        (conj marked worst)
                        (- swaps 1)
                        )
                    
                ))
        ))
    [initial_state] (set '()) swaps)) [8 7 6 5 4 3 2 1 0])
    )

#'user/get_random_state

In [2]:
(defn rbfs [initial_state, h]
    ((fn rbfs_in [states marked steps] 
    (let [best (apply min-key h states)]
        (if (solved best) steps 
            (
                let [successors (get_successors best)] 
                    (rbfs_in
                        (vec (filter (fn [e] (every? #(not= e %) marked)) (vec (concat successors (vec (remove (fn [e] (= e best)) states))))))
                        (conj marked best)
                        (+ 1 steps)
                        )
                    
                ))
        ))
    [initial_state] (set '()) 0))
(defn P [e enew ti] 
        (cond
            (< enew e) 1
            :else (/ 1 (+ 1 (/ (- enew e) ti)))
            ))
(defn T [x]
    (/ 1 (+ 1 x)))
(defn anneal [initial_state, e, kmax]
    (let [s initial_state] 
        ((fn inner[s e kmax k] 
             (cond
                 (= kmax k) [false k]
                 :else (let [snew (rand-nth (get_successors s))] 
                     (cond
                         (= 0 (e snew)) [true (+ 1 k)]
                         (> (P (e s) (e snew) (T (/ k kmax))) (rand)) (inner snew e kmax (+ 1 k))
                         :else (inner s e kmax (+ 1 k))
                         ))
                 ))
         s e kmax 0
         )))

#'user/anneal

In [3]:
(anneal (get_random_state 30) h1 1000)

[true 6]